# LifeSnaps Preprocessing Experiments

Expermiments amb un menor nombre de features.

#### Import de llibreries necessaries

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Imbalanced data pipeline
from imblearn.pipeline import Pipeline as ImbPipeline

# Core utilities
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

# Model definitions fora dels defints
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


############## IMPORTS DEL NOSTRE PAQUET ###################
# Obtenim les funcions per entrenar i evaluar els models i registrar les mètriques
from ai_health_assistant.utils.train_helpers import train_models, append_results, plot_learning_curve, mat_confusio, update_experiments_file

# Obtenim els classificadors i els seus parametres
from ai_health_assistant.utils.model_config import get_classifier_config, PARAM_GRIDS, CLASSIFIERS, BALANCING_METHODS

# Obtenim el target, features el la construcció del preprocessador
from ai_health_assistant.utils.prep_helpers import TARGET, build_preprocessor, FEATURES

# Configuració de pandas
pd.set_option('display.max_columns', None)

# Carrega de dades, netes i amb fe aplicat
df_train = pd.read_csv('../data/df_engineered_train.csv')
df_test = pd.read_csv('../data/df_engineered_test.csv')
    
print(f"Shape: {df_train.shape}")
print(f"Shape: {df_test.shape}")


Shape: (1832, 40)
Shape: (458, 40)


## Lectura de dades i split de train / test

### Train / Test Split

In [2]:
# Fem l'split de les dades, separant les features i el target
X_train = df_train[FEATURES]
y_train = df_train[TARGET]

X_test = df_test[FEATURES]
y_test = df_test[TARGET]

print(f"\nTrain shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")
print('\n','--'*50)
print(f"\nDistribució train:\n{y_train.value_counts(normalize=True)}")
print(f"\nDistribució test:\n{y_test.value_counts(normalize=True)}")

numerical_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=['number']).columns.tolist()

print(f"\nCol. numeriques ({len(numerical_features)}): \n{numerical_features}")
print(f"Col. categoriques ({len(categorical_features)}): \n{categorical_features}")



Train shape: (1832, 37)
Test shape: (458, 37)

 ----------------------------------------------------------------------------------------------------

Distribució train:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Distribució test:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Col. numeriques (35): 
['bmi', 'calories', 'steps', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes', 'resting_hr', 'minutes_below_default_zone_1', 'minutes_in_default_zone_1', 'minutes_in_default_zone_2', 'minutes_in_default_zone_3', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'sleep_deep_ratio', 'sleep_light_ratio', 'sleep_rem_ratio', 'sleep_wake_ratio', 'daily_temperature_variation', 'rmssd', 'spo2', 'full_sleep_breathing_rate', 'wake_after_sleep_pct', 'steps_norm_cal', 'deep_sleep_score', 'active_sedentary_ratio', 'sleep_activity_balance', 'bmi_hr_interaction', 'sleep_quality_index', 'hr_zone_variab

### Definim el preprocessador

In [3]:
preprocessor = build_preprocessor(df_train, FEATURES)

## ENTRENAMENT DEL MODEL BASE

In [8]:
display_cols = ['Experiment', 'Train F1 (1)', 'Train F1 (macro global)','Train Accuracy', 'Test Recall (1)', 'Test Precision (1)', 'Test F1 (1)', 'Test F1 (macro global)','Test Accuracy']

# --------------------------------------------------------------
# Filtre de selecció dels models a entrenar
PROVA_MODELS = ["BalancedRandomForest"]
# Selecció del mètode de balanceig
balance_name = "SMOTETomek"
# --------------------------------------------------------------

# Filtre per no haver de entrenar tots el models
CLASSIFIERS_FILTER = {k: v for k, v in CLASSIFIERS.items() if k in PROVA_MODELS}
balance_method = BALANCING_METHODS[balance_name]

### Regressió Logistica

In [5]:

reg_results = []
reg_models = {}

reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=2000,            # convergència assegurada
        class_weight="balanced", # tracta l’imbalance de la classe 1
        solver="lbfgs",          # ràpid i estable per datasets petits/mitjans
    ))
])

reg_param_grid = {
    "classifier__C": [0.001, 0.01, 0.1, 1, 10, 100, 200, 500, 1000]
}

best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train, 
        y_train,
        X_test,
        y_test, 
        reg_pipeline, 
        reg_param_grid,
        search_type='grid',
    )

reg_results_df = append_results(
    reg_results,
    "LogisticRegression",
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="EntrenamentBasic"
)

display(reg_results_df[display_cols])
update_experiments_file(reg_results_df)

Entrenant model...

Train F1 (1): 0.5156 | Test F1 (1): 0.4800 | Train Acc: 0.5939 | Test Acc: 0.5742
              precision    recall  f1-score   support

         0.0     0.6680    0.6135    0.6396       282
         1.0     0.4523    0.5114    0.4800       176

    accuracy                         0.5742       458
   macro avg     0.5601    0.5624    0.5598       458
weighted avg     0.5851    0.5742    0.5782       458



,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LogisticRegression_EntrenamentBasic,0.51562,0.583,0.59389,0.51136,0.45226,0.48,0.55978,0.57424



Métriques guardades a ../results/02_experiments/experiments.csv



In [9]:
base_results = []

for model, classifier in CLASSIFIERS_FILTER.items():

    if model == "BalancedRandomForest_PROVA":
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("classifier", classifier)
        ])
    else:
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("balance", balance_method),
            ("classifier", classifier)
        ])

    print(f"\n==== {model} ====")
    best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train,
        y_train,
        X_test,
        y_test, 
        pipeline,
        PARAM_GRIDS[model]
    )

    base_results_df = append_results(
    base_results,
    model,
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="EntrenamentBasic"
)


==== BalancedRandomForest ====
Entrenant model...

Train F1 (1): 0.8101 | Test F1 (1): 0.5029 | Train Acc: 0.8564 | Test Acc: 0.6245
              precision    recall  f1-score   support

         0.0     0.6910    0.7057    0.6982       282
         1.0     0.5118    0.4943    0.5029       176

    accuracy                         0.6245       458
   macro avg     0.6014    0.6000    0.6006       458
weighted avg     0.6221    0.6245    0.6232       458



In [10]:
update_experiments_file(base_results_df)


Métriques guardades a ../results/02_experiments/experiments.csv



## EXPERIMENT 1: Importancia de les caracteristiques

Entrenem RandomForest per indentificar les caracteristiques més importants (3 i 5), posteriorment entrenem els models utilitzant aquestes 3 i 5 característiques, per veure si augmenta el rendiment del model. Proavarem tambe amb permutation importances.

### Feature Importance

La Gini importance d’una feature és: La suma de totes les reduccions d’impuresa (Gini) que ha causat al llarg de tots els arbres i de totes les seves aparicions.

In [11]:
# Param grid & pipeline bàsic de random forest
rf_name = "BalancedRandomForest"

pipeline = ImbPipeline([
    ("preprocessor", preprocessor),
    ("classifier", CLASSIFIERS[rf_name])
])

# Entrenament del model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipeline, 
    PARAM_GRIDS[rf_name],
    search_type='grid'
)
preprocessor = best_est.named_steps['preprocessor']

feature_names = preprocessor.get_feature_names_out()

# Serie em els noms
importances_raw = pd.Series(
    best_est.named_steps["classifier"].feature_importances_,
    index=feature_names
)

agg_importances = (
    importances_raw.groupby(feature_names).sum().sort_values(ascending=False)
)

# Top-10 i Top-15 importàncies

top3 = agg_importances.head(3).index.tolist() 
top5 = agg_importances.head(5).index.tolist()
print("Top-10 features:", top3)
print("Top-15 features:", top5)

# Visualització de les Top-15 importàncies
plt.figure(figsize=(10, 6))
plt.bar(top5, agg_importances.head(5).values)
plt.xticks(rotation=45, ha="right")
plt.title("Top-15 Gini Importances")
plt.xlabel("Feature")
plt.ylabel("Importància (Gini)")
plt.tight_layout()
plt.show()


Entrenant model...

Train F1 (1): 0.7096 | Test F1 (1): 0.5558 | Train Acc: 0.6976 | Test Acc: 0.5218
              precision    recall  f1-score   support

         0.0     0.7234    0.3617    0.4823       282
         1.0     0.4322    0.7784    0.5558       176

    accuracy                         0.5218       458
   macro avg     0.5778    0.5701    0.5190       458
weighted avg     0.6115    0.5218    0.5105       458

Top-10 features: ['num__calories', 'num__bmi', 'num__resting_hr']
Top-15 features: ['num__calories', 'num__bmi', 'num__resting_hr', 'num__bmi_hr_interaction', 'num__daily_temperature_variation']


### Rentrenament de models 10-15 millors features

In [ ]:
# Definim els dos conjunts de features
feature_sets = {
    "Top3": top3,   # llista de 3 noms de feature “base”
    "Top5": top5    # llista de 5 noms de feature “base”
}

# Reentrenament i avaluació per a cada subset
fi_results = []
preprocessor = build_preprocessor(X_train, FEATURES)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Generem el train transformat i el test transformat
X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)


for label, feats in feature_sets.items():
    print(f"\nEntrenament models amb {label}")
    for model, classifier in CLASSIFIERS_FILTER.items():

        print(f'==== {model} ====')
        if model == "BalancedRandomForest": # Em dona millors resutats sense balance
            pipe = ImbPipeline([
                ("classifier", classifier)
            ])
        else:
            pipe = ImbPipeline([
                ("balance", balance_method),
                ("classifier", classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
            X_train_transformed[feats],
            y_train,
            X_test_transformed[feats],
            y_test,
            pipe,
            PARAM_GRIDS[model]
        )

        fi_results_df =append_results(
            fi_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"FI_{label}"
        )


Entrenament models amb Top3
==== MLP ====
Entrenant model...

Train F1 (1): 0.5234 | Test F1 (1): 0.4898 | Train Acc: 0.5557 | Test Acc: 0.5087
              precision    recall  f1-score   support

         0.0     0.6477    0.4433    0.5263       282
         1.0     0.4075    0.6136    0.4898       176

    accuracy                         0.5087       458
   macro avg     0.5276    0.5284    0.5081       458
weighted avg     0.5554    0.5087    0.5123       458

==== SVM ====
Entrenant model...

Train F1 (1): 0.5233 | Test F1 (1): 0.5143 | Train Acc: 0.5306 | Test Acc: 0.5175
              precision    recall  f1-score   support

         0.0     0.6704    0.4255    0.5206       282
         1.0     0.4194    0.6648    0.5143       176

    accuracy                         0.5175       458
   macro avg     0.5449    0.5452    0.5174       458
weighted avg     0.5739    0.5175    0.5182       458

==== GradientBoosting ====
Entrenant model...

Train F1 (1): 0.5161 | Test F1 (1): 0.

In [33]:
display(fi_results_df[display_cols])
update_experiments_file(fi_results_df)

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,MLP_FI_Top3,0.52342,0.55363,0.55568,0.61364,0.40755,0.48980,0.50806,0.50873
1,SVM_FI_Top3,0.52328,0.53046,0.53057,0.66477,0.41935,0.51429,0.51745,0.51747
2,GradientBoosting_FI_Top3,0.51613,0.58674,0.59880,0.52841,0.45146,0.48691,0.55993,0.57205
3,BalancedRandomForest_FI_Top3,0.63520,0.58999,0.59498,0.78977,0.43302,0.55936,0.51834,0.52183
4,LGBM_FI_Top3,0.63000,0.59264,0.59607,0.71023,0.38820,0.50201,0.45435,0.45852
5,MLP_FI_Top5,0.51058,0.58336,0.59607,0.51705,0.42326,0.46547,0.53369,0.54367
6,SVM_FI_Top5,0.47796,0.56764,0.58624,0.47727,0.43750,0.45652,0.54578,0.56332
7,GradientBoosting_FI_Top5,0.54923,0.61962,0.63264,0.52273,0.46231,0.49067,0.56881,0.58297
8,BalancedRandomForest_FI_Top5,0.67946,0.66173,0.66266,0.79545,0.44872,0.57377,0.54389,0.54585
9,LGBM_FI_Top5,0.64659,0.61277,0.61572,0.74432,0.40308,0.52295,0.47353,0.47817



Métriques guardades a ../results/02_experiments/experiments.csv



### Permutation Importance

Per avaluar la importància de les característiques del model. Serveix per determinar quines característiques tenen més impacte en el rendiment del model.

In [34]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Train el random forest
rf_name = 'BalancedRandomForest'
clf_rf, param_grid_rf = get_classifier_config(rf_name)

# nou pipeline
pipe = ImbPipeline([
    ("preprocessor", preprocessor),
    ("classifier", clf_rf)
])

# Train el model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipe, 
    param_grid_rf,
    search_type='grid'
)

best_rf_model = best_est

preprocessor = best_rf_model.named_steps['preprocessor']
classifier = best_rf_model.named_steps['classifier']

X_test_transformed = preprocessor.transform(X_test)

feature_names = preprocessor.get_feature_names_out()

# pipeline nomes classifier
final_estimator = Pipeline([
    ('classifier', classifier)
])

# Calculem la importància permutativa
result = permutation_importance(
    final_estimator,
    X_test_transformed, 
    y_test,
    n_repeats=200,
    n_jobs=-1,
    random_state=42
)

# Creem Series amb els noms de les caracteristiques
perm_importances = pd.Series(
    result.importances_mean, 
    index=feature_names
).sort_values(ascending=False)

# Top 3 i 5 features
perm_top_features = {
    3: perm_importances.head(3).index.tolist(),
    5: perm_importances.head(5).index.tolist()
}

print("\nTop-15 features (Permutation):")
display(perm_importances.head(15).to_frame("Importancia"))

# Plot
plt.figure(figsize=(12, 6))
top15 = perm_importances.head(15)
bars = plt.barh(top15.index, top15.values)
plt.title("Top-15 Permutation Importance", fontsize=14)
plt.xlabel("Mean Decrease in F1 Score", fontsize=12)
plt.gca().invert_yaxis()  # Most important features on top
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Add value labels on the bars
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.001, bar.get_y() + bar.get_height()/2., 
             f'{width:.3f}', 
             ha='left', va='center')

plt.tight_layout()
plt.show()

Entrenant model...

Train F1 (1): 0.7096 | Test F1 (1): 0.5558 | Train Acc: 0.6976 | Test Acc: 0.5218
              precision    recall  f1-score   support

         0.0     0.7234    0.3617    0.4823       282
         1.0     0.4322    0.7784    0.5558       176

    accuracy                         0.5218       458
   macro avg     0.5778    0.5701    0.5190       458
weighted avg     0.6115    0.5218    0.5105       458


Top-15 features (Permutation):


,Importancia
num__bmi,0.017140
num__calories,0.016321
num__recovery_factor,0.014389
num__full_sleep_breathing_rate,0.014017
num__minutesAsleep,0.010764
num__steps_norm_cal,0.010131
num__steps,0.005251
num__daily_temperature_variation,0.005164
num__sleep_eff_rmssd,0.005011
num__wake_after_sleep_pct,0.003777


### Reentrenament Permutation importance

In [35]:
# Definim els dos conjunts de features basats en importància de permutació
perm_sets = {
    "Perm_Top3": perm_top_features[3],  # Índexos de les 10 millors característiques
    "Perm_Top5": perm_top_features[5]   # Índexos de les 15 millors característiques
}

print(perm_top_features)

# Reentrenament i avaluació per a cada subset
perm_results = []

# Preprocessem un cop fora del bucle per eficiència
preprocessor = build_preprocessor(X_train, FEATURES)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Convertim a DataFrame mantenint els índexs
feature_names = preprocessor.get_feature_names_out()
X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)

for label, feat_indices in perm_sets.items():
    print(f"\nEntrenament models amb {label}")
    
    for model_name, classifier in CLASSIFIERS_FILTER.items():
        print(f'==== {model_name} ====')
        
        if model_name == "BalancedRandomForest":
            pipe = ImbPipeline([
                ("classifier", classifier)
            ])
        else:
            pipe = ImbPipeline([
                ("balance", balance_method),
                ("classifier", classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
            X_train_transformed[feat_indices],
            y_train,
            X_test_transformed[feat_indices],
            y_test,
            pipe,
            PARAM_GRIDS[model_name]
        )

        perm_results_df = append_results(
            perm_results,
            model_name,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=label  # Usem directament l'etiqueta (Perm_Top10 o Perm_Top15)
        )

{3: ['num__bmi', 'num__calories', 'num__recovery_factor'], 5: ['num__bmi', 'num__calories', 'num__recovery_factor', 'num__full_sleep_breathing_rate', 'num__minutesAsleep']}

Entrenament models amb Perm_Top3
==== MLP ====
Entrenant model...

Train F1 (1): 0.4444 | Test F1 (1): 0.4294 | Train Acc: 0.5961 | Test Acc: 0.5939
              precision    recall  f1-score   support

         0.0     0.6558    0.7163    0.6847       282
         1.0     0.4667    0.3977    0.4294       176

    accuracy                         0.5939       458
   macro avg     0.5613    0.5570    0.5571       458
weighted avg     0.5831    0.5939    0.5866       458

==== SVM ====
Entrenant model...

Train F1 (1): 0.4977 | Test F1 (1): 0.4570 | Train Acc: 0.5901 | Test Acc: 0.5590
              precision    recall  f1-score   support

         0.0     0.6527    0.6064    0.6287       282
         1.0     0.4337    0.4830    0.4570       176

    accuracy                         0.5590       458
   macro avg    

In [36]:
display(perm_results_df[display_cols])

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,MLP_Perm_Top3,0.44444,0.56356,0.59607,0.39773,0.46667,0.42945,0.55710,0.59389
1,SVM_Perm_Top3,0.49766,0.57571,0.59007,0.48295,0.43367,0.45699,0.54283,0.55895
2,GradientBoosting_Perm_Top3,0.54827,0.59591,0.60153,0.55682,0.46890,0.50909,0.57658,0.58734
3,BalancedRandomForest_Perm_Top3,0.63749,0.59824,0.60207,0.80114,0.44062,0.56855,0.52951,0.53275
4,LGBM_Perm_Top3,0.61762,0.59601,0.59716,0.71591,0.40909,0.52066,0.49181,0.49345
5,MLP_Perm_Top5,0.55452,0.57932,0.58079,0.64205,0.45020,0.52927,0.55912,0.56114
6,SVM_Perm_Top5,0.53171,0.57613,0.58079,0.58523,0.44978,0.50864,0.55960,0.56550
7,GradientBoosting_Perm_Top5,0.55717,0.61016,0.61736,0.56250,0.47596,0.51562,0.58300,0.59389
8,BalancedRandomForest_Perm_Top5,0.65528,0.63538,0.63646,0.78409,0.45246,0.57380,0.55127,0.55240
9,LGBM_Perm_Top5,0.63559,0.60000,0.60317,0.73295,0.41613,0.53086,0.50032,0.50218


In [37]:
update_experiments_file(perm_results_df)


Métriques guardades a ../results/02_experiments/experiments.csv



## EXPERIMENT 2: PCA


Es realitza una anàlisi de components principals (PCA) per examinar com evolucionen els components més rellevants del conjunt de dades en termes de variància explicada acumulada, considerant els primers 3, 4, 5, 6, 7, 8, 9, 10 components

In [ ]:
pca = PCA(random_state=42)
pca.fit(X_train_transformed)

explained_cumsum = pca.explained_variance_ratio_.cumsum()*100

ks = [3, 4, 5, 6, 7, 8, 9, 10]
cums = explained_cumsum[[k-1 for k in ks]]

plt.figure(figsize=(8, 4))
plt.plot(
    range(1, len(explained_cumsum) + 1),
    explained_cumsum,
)
plt.scatter(ks, cums)
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Evolución de la varianza explicada según n_components')
plt.grid(True)
plt.tight_layout()
plt.show()

In [39]:
n_components_list = [3, 4, 5, 6, 7, 8, 9, 10]
for k in n_components_list:
    # Ajusta PCA
    pca = PCA(n_components=k, random_state=42)
    pca.fit(X_train_transformed)

    # loadings: matriz (n_features, k)
    loadings = pca.components_.T

    # importancia = suma de cargas absolutes de cada feature en tots els components
    importance = np.sum(np.abs(loadings), axis=1)

    # crea DataFrame, ordena top-k
    df_imp = pd.DataFrame({
        'feature':    feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    topk = df_imp.head(k)

    plt.figure()
    plt.barh(topk['feature'][::-1], topk['importance'][::-1])
    plt.xlabel('Importancia (suma de |carregues|)')
    plt.title(f'Top {k} features segons PCA')
    plt.tight_layout()
    plt.show()

In [40]:
# Guarda resultats i models
pca_results = []

for k in n_components_list:
    print(f"\n-- PCA - {k} components --")

    for model, classifier in CLASSIFIERS_FILTER.items():
        if model == "BalancedRandomForest":
            pipeline = ImbPipeline([
                ("preprocessor", preprocessor),
                ("pca",          PCA(n_components=k, random_state=42)),
                ("classifier",   classifier)
            ])
        else:
            pipeline = ImbPipeline([
                ("preprocessor", preprocessor),
                ("balance",       balance_method),
                ("pca",          PCA(n_components=k, random_state=42)),
                ("classifier",   classifier)
            ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
        X_train, 
        y_train,
        X_test,
        y_test,
        pipeline, 
        PARAM_GRIDS[model],
        search_type="grid"
        )

        df_pca_results = append_results(
            pca_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"PCA_{k}",
        )



-- PCA - 3 components --
Entrenant model...

Train F1 (1): 0.4928 | Test F1 (1): 0.4853 | Train Acc: 0.5606 | Test Acc: 0.5786
              precision    recall  f1-score   support

         0.0     0.6718    0.6170    0.6433       282
         1.0     0.4573    0.5170    0.4853       176

    accuracy                         0.5786       458
   macro avg     0.5646    0.5670    0.5643       458
weighted avg     0.5894    0.5786    0.5826       458

Entrenant model...

Train F1 (1): 0.4976 | Test F1 (1): 0.5039 | Train Acc: 0.5437 | Test Acc: 0.5786
              precision    recall  f1-score   support

         0.0     0.6816    0.5922    0.6338       282
         1.0     0.4601    0.5568    0.5039       176

    accuracy                         0.5786       458
   macro avg     0.5709    0.5745    0.5688       458
weighted avg     0.5965    0.5786    0.5839       458

Entrenant model...

Train F1 (1): 0.5237 | Test F1 (1): 0.4114 | Train Acc: 0.6059 | Test Acc: 0.5502
              

In [41]:
display(df_pca_results[display_cols])
update_experiments_file(df_pca_results)

,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,MLP_PCA_3,0.49275,0.55259,0.56059,0.51705,0.45729,0.48533,0.56429,0.57860
1,SVM_PCA_3,0.49760,0.53980,0.54367,0.55682,0.46009,0.50386,0.56882,0.57860
2,GradientBoosting_PCA_3,0.52375,0.59381,0.60590,0.40909,0.41379,0.41143,0.52374,0.55022
3,BalancedRandomForest_PCA_3,0.61726,0.51676,0.53766,0.84659,0.40822,0.55083,0.45142,0.46943
4,LGBM_PCA_3,0.64516,0.60578,0.60972,0.74432,0.41066,0.52929,0.48792,0.49127
5,MLP_PCA_4,0.54714,0.57124,0.57260,0.63636,0.44800,0.52582,0.55679,0.55895
6,SVM_PCA_4,0.49384,0.56281,0.57369,0.50000,0.45596,0.47696,0.56207,0.57860
7,GradientBoosting_PCA_4,0.54631,0.61768,0.63100,0.43750,0.41622,0.42659,0.52681,0.54803
8,BalancedRandomForest_PCA_4,0.66600,0.63425,0.63701,0.79545,0.41420,0.54475,0.48133,0.48908
9,LGBM_PCA_4,0.65806,0.62012,0.62391,0.75000,0.39640,0.51866,0.45835,0.46507



Métriques guardades a ../results/02_experiments/experiments.csv



## Anàlisi de resultats

In [42]:
# Llegeix la taula d'experiments (ja ordenada per F1)
experiments_df = pd.read_csv("../results/02_experiments/experiments.csv")
top5 = experiments_df.head(5)

# Mostra els top 5
display(top5)


,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Precision (1),Test Recall (1),Test F1 (1),Test F1 (macro global),Test Accuracy,Best Params
0,BalancedRandomForest_Perm_Top5,0.65528,0.63538,0.63646,0.45246,0.78409,0.57380,0.55127,0.55240,"{'classifier__n_estimators': 1021, 'classifier..."
1,BalancedRandomForest_FI_Top5,0.67946,0.66173,0.66266,0.44872,0.79545,0.57377,0.54389,0.54585,NaN
2,LGBM_EntrenamentBasic,0.88122,0.89912,0.90229,0.51111,0.65341,0.57357,0.62076,0.62664,NaN
3,BalancedRandomForest_Perm_Top3,0.63749,0.59824,0.60207,0.44062,0.80114,0.56855,0.52951,0.53275,"{'classifier__n_estimators': 1021, 'classifier..."
4,BalancedRandomForest_PCA_8,0.67928,0.65620,0.65775,0.43114,0.81818,0.56471,0.50895,0.51528,{'classifier__class_weight': 'balanced_subsamp...
